## prompt
```
Implement a function in Python with following properties:

Function name: find_eventual_safe_states
Description
-----------

Given a directed graph of n nodes, where node u in {0, 1, ..., n-1}
. Find all safe node x, where all paths starting from x eveually lead to a terminal node (node without outgoing edges) or another safe node. 

Parameters
----------
graph : List[List[int]]
    adjacent list of directed graph

Returns
-------
results : List[int]
    list of all safe nodes, sorted in ascending order
```

## deep-thinking

Okay, I need to solve this problem where I have to find all the safe nodes in a directed graph. The safe nodes are those where every possible path starting from them eventually leads to a terminal node (a node with no outgoing edges) or another safe node. Hmm, right. So, how do I approach this?

Let me think about the problem. A terminal node is obviously safe because it has no outgoing edges. So any node that can only reach terminal nodes through all paths is safe. But there's also the case where a node might reach another safe node. So the safe nodes form a kind of set where their paths end in terminals or in each other.

Wait, this reminds me of cycles. Because if a node is part of a cycle, then there's a path that loops indefinitely, so those nodes wouldn't be safe. So, the problem is essentially about finding nodes that are not part of any cycles and all their paths eventually exit the cycle. Or maybe not just cycles, but any path that leads to a cycle. Because if a node's path can enter a cycle, then it's not safe.

So, the key idea is to find all nodes that are not part of any cycle and also are not in a path that leads into a cycle. Because if a node's path can reach a cycle, then there's a way for it to loop forever, which would make it unsafe.

So, how do I detect this? One approach is to find all nodes that are in cycles or can reach a cycle. Then the safe nodes are those that are not in this set.

Alternatively, maybe we can model this using some kind of state for each node. Like, during a traversal, we can mark nodes as safe or unsafe. For a node to be safe, all of its outgoing edges must lead to nodes that are also safe. So, it's a bit like topological sorting but in reverse.

Wait, topological sorting is for DAGs. But here the graph might have cycles. So maybe we need a way to determine for each node whether all possible paths from it eventually reach a terminal node.

Another approach could be to reverse the graph and perform some kind of BFS from the terminal nodes. Because the terminal nodes are safe. Then, any node that can reach a terminal node via all paths would be safe. Wait, but how?

Alternatively, perhaps using a three-color marking system (like in cycle detection). Nodes are marked as unvisited (0), visiting (1), or visited (2). If during a DFS we detect a cycle (i.e., we encounter a node that is in the visiting state), then all nodes in that cycle are unsafe. But maybe this isn't sufficient because even if a node isn't in a cycle, but can reach a cycle, then it's also unsafe.

Wait, no. Let's think: a node is safe if all paths from it eventually reach a terminal. So if a node is in a cycle, or can reach a cycle, then it's not safe. Because cycles can't reach terminals. So the safe nodes are exactly those that are not part of any cycle and all paths from them do not enter any cycles.

So how do we find nodes that are not in any cycle and are in the part of the graph where all paths lead to terminals without cycles.

Hmm. So perhaps we can model this with a modified DFS that checks for each node whether all paths from it terminate. But how to track that.

Another idea: a node is safe if all of its children are safe. Because if all children of a node are safe, then any path from the node must go through a child, which is safe, meaning that path eventually leads to a terminal. So, this sounds like a post-order traversal where we can determine if a node is safe based on its children.

But how to handle cycles here. Let's see. For example, if a node points to a node that is part of a cycle, then that node is not safe, so the original node is not safe either.

So maybe the approach is to process the nodes in such a way that once a node is determined to be safe, we can mark it and then check if any of its parents (nodes that point to it) can also be marked as safe if all their children are safe.

This is similar to Kahn's algorithm for topological sorting. Kahn's algorithm works by processing nodes with in-degree zero. But here, perhaps we can process nodes where all outgoing edges lead to safe nodes. So, in reverse.

Wait, here's a possible approach:

1. The terminal nodes (those with no outgoing edges) are safe. So we can start by adding all terminal nodes to the safe set.

2. For any node that only points to nodes in the safe set, that node is also safe. We can add it to the safe set.

3. Repeat this process until no more nodes can be added.

But how do we efficiently check this? Because for a node to be safe, all of its outgoing edges must point to nodes that are already in the safe set.

This sounds like a reverse BFS approach. So the algorithm could be:

- Initialize a queue with all terminal nodes.

- For each node in the queue, check all nodes that point to it (since we need to process in reverse). Wait, no. Wait, in this approach, the direction is reversed. Let's see.

Wait, let's think in reverse. Let's build a reverse graph where each edge is reversed. Then, the nodes that are safe in the original graph correspond to nodes that can only reach the terminal nodes. But perhaps using the reverse graph, the safe nodes are those that can be reached by starting from the terminal nodes and following the reverse edges, but ensuring that all outgoing edges (in the original graph) lead to safe nodes.

Alternatively, here's another way: For each node, if all its outgoing edges point to nodes that are already known to be safe, then the node is safe. So we can process nodes in such a way. This is similar to a topological sort where we process nodes in order of dependencies.

So the algorithm steps could be:

- Identify all terminal nodes (nodes with no outgoing edges) and mark them as safe.

- For each node that is not yet marked safe, check if all of its outgoing edges point to nodes that are marked as safe. If yes, mark this node as safe.

- Repeat this process until no more nodes can be marked as safe.

But how to efficiently implement this. Because each time a node is marked as safe, we need to check all nodes that have edges pointing to it (or perhaps nodes that have edges pointing from them to this node?), wait no. Let me clarify.

Wait, for a node to be safe, all its outgoing edges must be to safe nodes. So when a new node is marked as safe, we need to check all nodes that have edges pointing to this new safe node to see if all their outgoing edges are now pointing to safe nodes.

Wait, no. Let's say node X has an outgoing edge to node Y. If Y becomes safe, then X's outgoing edge to Y is now safe, but X can only be marked as safe if all of its outgoing edges point to safe nodes. So, for each node X, once one of its outgoing edges points to a safe node, that's a step towards X possibly being safe. But X can't be marked as safe until all of its outgoing edges are to safe nodes.

So the way to model this is to track, for each node, the number of outgoing edges that are not yet known to be safe. Once this count reaches zero, the node is safe.

So here's an approach inspired by Kahn's algorithm:

1. For each node, compute the set of outgoing edges.

2. For each node, maintain a count of how many of its outgoing edges point to nodes that are not yet safe. Let's call this the "unsafe outdegree".

3. Initialize all terminal nodes (nodes with zero outgoing edges) as safe. For these nodes, their unsafe outdegree is zero.

4. Create a queue and add all terminal nodes to it.

5. For each node processed from the queue:

a. Iterate over all nodes that have an incoming edge to this node (i.e., in the reversed graph). For each such node X, decrement X's unsafe outdegree by 1 (since one of its outgoing edges now points to a safe node).

b. If X's unsafe outdegree becomes zero, then X is now safe. ==Add it to the queue==<NOTE: should be remove it from the queue>.

6. Continue until the queue is empty.

7. All nodes that have been processed (those in the queue) are safe. The rest are not.

This seems promising. Let's think through an example.

Suppose the graph is [[1,2],[2],[]]. Then node 2 is terminal (outgoing edges are zero). So initially, node 2 is added to the queue.

Processing node 2:

Look at all nodes that point to node 2. In the original graph, node 0 points to 2 via index 1 (graph[0] is [1,2]), node 1 points to 2 via index 0 (graph[1] is [2]). So in the reversed graph, node 2's incoming edges are from 0 and 1.

So for each of these nodes (0 and 1), we decrement their unsafe outdegree.

Wait, but for node 1, its outgoing edges are [2]. So when node 2 is processed, node 1's unsafe outdegree is 1 (since it has one outgoing edge to node 2, which was initially not safe). Wait, no. Wait, initially, the unsafe outdegree for each node is the number of outgoing edges. For terminal nodes, the unsafe outdegree is 0. For node 1, which has one outgoing edge to 2 (which is initially not processed, except for 2 is terminal). So when the queue starts with 2, which is a terminal node. Then, for node 1, which has an outgoing edge to 2 (which is safe), node 1's unsafe outdegree is initially 1 (since it has one outgoing edge). Then, when processing node 2, we look at all nodes that have edges pointing to 2 (in the original graph). So for node 1, which has an edge to 2, when we process node 2, we subtract 1 from node 1's unsafe outdegree. So node 1's count becomes 0. Then node 1 is added to the queue.

Processing node 1:

Now, look at all nodes that point to node 1. Node 0 has an edge to 1. So for node 0, its unsafe outdegree is 2 (since graph[0] is [1,2]). When node 1 is processed, we subtract 1 from node 0's count. So node 0's count becomes 1. Then, when node 2 was processed, node 0's count was also decremented by 1 (since node 2 is processed). Wait, maybe I need to adjust the initial approach.

Wait, the reversed graph would track for each node, which nodes have edges pointing to it. So, for each node, when it's added to the queue (marked as safe), all nodes that have edges leading into it (in the original graph) need to have their unsafe outdegree decremented. Because for those nodes, one of their outgoing edges now leads to a safe node.

So, in the example:

Original graph:

0 → 1, 2

1 → 2

2 → (nothing)

Reversed graph (for each node, who points to it):

node 0: no incoming edges.

node 1: incoming from 0.

node 2: incoming from 0 and 1.

Initially, the unsafe outdegree for each node:

node 0: 2 (edges to 1 and 2)

node 1: 1 (edge to 2)

node 2: 0 (terminal)

So queue starts with node 2.

Process node 2:

For each node that points to it (0 and 1), decrement their unsafe outdegree.

For node 0: unsafe_outdegree becomes 2-1=1.

For node 1: unsafe_outdegree becomes 1-1=0.

Since node 1's outdegree is now zero, add it to the queue.

Process node 1:

For each node that points to it (node 0), decrement their unsafe_outdegree.

Node 0's unsafe_outdegree was 1, now 1-1=0.

Add node 0 to the queue.

Process node 0:

For each node that points to it (none), so nothing to do.

Now the queue is empty. The safe nodes are 0,1,2. Wait, but node 0's edges are to 1 and 2, which are both safe. So yes, all paths from 0 go to 1 or 2, which are safe. So all three nodes are safe.

But what if there's a cycle?

Another example: graph is [[1], [2], [0]]. This is a cycle. So none of the nodes are safe. Because all paths from any node loop around the cycle and never reach a terminal.

So let's see how the algorithm works here.

Original graph:

0 → 1

1 → 2

2 → 0

Reversed graph:

node 0: incoming from 2

node 1: incoming from 0

node 2: incoming from 1

Unsafe_outdegree for each node:

node 0: 1 (edge to 1)

node 1: 1 (edge to 2)

node 2: 1 (edge to 0)

Terminal nodes are those with no outgoing edges. Here, none of the nodes are terminals. So initially, the queue is empty. So no processing happens, and all nodes remain unsafe. Which is correct.

Another example: graph [[], [0,2,3], [], [4], []]

So node 0 has no outgoing edges (terminal). Node 4 also is terminal.

So the safe nodes would be 0,4 <NOTE: mistake>, and any nodes that point only to them.

Let's see:

Initially, queue has 0 and 4.

Processing node 0: look at nodes that point to 0. Which is node 1. Decrement node 1's unsafe_outdegree (original outdegree is 3: edges to 0,2,3). So node 1's count was 3. After processing 0, it becomes 3-1=2. Not zero yet.

Processing node 4: look at nodes pointing to 4. That's node 3. Decrement node 3's count (original outdegree is 1 (edge to 4)). So node 3's count was 1. Now 0. So add node 3 to queue.

Process node 3: look at nodes pointing to 3. Which is node 1. Decrement node 1's count (current is 2) by 1. Now node 1's count is 1. Not zero.

Then queue is empty. So the safe nodes are 0,3,4.

Wait, but node 3's outgoing edge is to 4, which is safe. So node 3 is safe. Node 1 has edges to 0 (safe), 2 (not safe), and 3 (safe). So two safe, one not. So node 1's unsafe_outdegree is 1, so it's not safe. Node 2's outgoing edges are none (terminal?) <NOTE: nice, mistake noticed>, wait no. The graph for node 2 is empty (original graph entry for 2 is []?), so node 2 is a terminal. Wait, in the given example, the graph is [[], [0,2,3], [], [4], []].

So nodes 0: [], 1: [0,2,3], 2: [], 3: [4], 4: [].

So nodes 0,2,4 are terminals. So the queue starts with 0,2,4.

Processing node 0:

Look at nodes that point to 0 (node 1). Decrement node 1's unsafe_outdegree from 3 (original outedges: 0,2,3) to 2.

Processing node 2:

Look at nodes that point to 2 (node 1). Decrement node 1's unsafe_outdegree from 2 to 1.

Processing node 4:

Look at nodes that point to 4 (node 3). Decrement node 3's unsafe_outdegree from 1 (outedge to 4) to 0. So node 3 is added to the queue.

Process node 3:

Look at nodes pointing to 3 (node 1). Decrement node 1's unsafe_outdegree from 1 to 0. So node 1 is added to queue.

Process node 1:

Look at nodes pointing to 1 (none?), because in the original graph, node 1 is pointed by no one. So after processing, node 1's unsafe_outdegree is zero. So when node 1 is processed, are there any nodes pointing to it? If not, nothing happens. So the final safe nodes are 0,2,4,3,1.

Wait, but node 1 has an edge to 3 (safe) and 0 (safe) and 2 (safe). So all of its outgoing edges are to safe nodes. So node 1 is safe. So yes, all nodes in this case except maybe node 2?

Wait, node 2 is a terminal. So yes, all nodes except those in cycles.

Wait in this example, node 1's outedges are all to safe nodes. So it should be safe. So the algorithm's result would be [0,1,2,3,4], which is correct.

So the algorithm works.

So the steps are:

1. Create a reversed adjacency list: for each node, track which nodes point to it (i.e., for each edge u→v in original graph, add u to reversed_adj[v]).

2. Compute the initial unsafe_outdegree for each node: the number of outgoing edges. For nodes with zero, they are terminal and added to the queue.

3. Then, process nodes in the queue. For each node, iterate through reversed_adj (those nodes that have edges to this node in the original graph). For each such node u, decrement their unsafe_outdegree by 1. If after decrementing, their unsafe_outdegree becomes zero, add them to the queue.

4. The nodes processed in this way are the safe nodes.

Because, a node is added to the queue only when all of its outgoing edges lead to nodes that are already safe (i.e., their unsafe_outdegree is zero). Therefore, when a node is processed, all its outgoing edges point to safe nodes, so it itself is safe.

So the implementation steps:

- For each node in the graph, compute the outgoing edges.

- Build reversed adjacency list.

- Compute the outdegree (unsafe_outdegree) for each node.

- Initialize the queue with nodes that have outdegree zero.

- Process each node in the queue, and for each neighbor in the reversed adjacency list, decrease their outdegree. If their outdegree becomes zero, add to the queue.

- The safe nodes are all nodes that were processed (i.e., those that were added to the queue and processed).

Wait, but the reversed adjacency list is built based on the original edges. So for each node v, reversed_adj[v] is a list of nodes u such that u has an edge to v.

So the code steps:

In Python:

def find_eventual_safe_states(graph):

n = len(graph)

outdegree = [0] * n

reversed_adj = [[] for _ in range(n)]

for u in range(n):

outdegree[u] = len(graph[u])

for v in graph[u]:

reversed_adj[v].append(u)

queue = []

for u in range(n):

if outdegree[u] == 0:

queue.append(u)

safe = [False] * n

while queue:

v = queue.pop(0)

safe[v] = True

for u in reversed_adj[v]:

outdegree[u] -= 1

if outdegree[u] == 0:

queue.append(u)

result = [u for u in range(n) if safe[u]]

return sorted(result)

Yes, this seems correct.

Let's test this code with the example I had earlier:

Test case 1:

graph = [[1,2],[2],[]]

n=3.

reversed_adj:

for v=0: reversed_adj[0] is empty.

for v=1: nodes u that point to 1. From original graph, u=0 (since graph[0] is [1,2]. So reversed_adj[1] = [0].

for v=2: nodes u that point to 2. From graph[0] →2, graph[1]→2. So reversed_adj[2] = [0,1].

outdegree is:

node 0: 2 (edges to 1 and 2)

node 1: 1 (edge to 2)

node 2: 0.

queue starts with node 2.

Processing node 2: set safe[2] = True.

Then, for each u in reversed_adj[2] (u=0 and 1).

For u=0: outdegree[0] becomes 2-1=1.

For u=1: outdegree[1] becomes 1-1=0. Add to queue.

Now queue has [1].

Processing node 1: set safe[1] = True.

Check reversed_adj[1] (u=0).

outdegree[0] becomes 1-1=0. Add to queue.

Processing node 0: set safe[0] = True.

Now all nodes are safe, so result is [0,1,2], which is correct.

Another test case with cycle: graph [[1],[2],[0]]

n=3.

reversed_adj:

for v=0: u=2.

v=1: u=0.

v=2: u=1.

outdegree: all 1.

queue is empty. So no processing. The result is empty list. Correct.

Another example where some nodes are safe and others not:

graph = [[1,2],[],[3],[],[]]

Nodes 3 and 4 are terminal (outdegree zero).

Wait, node 4's graph entry is empty, so it's a terminal.

But the graph is:

0 →1,2

1 → nothing (terminal?)

Wait, original graph as per code: graph is a list of lists. For graph = [[1,2], [], [3], [], []], the nodes are:

0: [1,2]

1: []

2: [3]

3: []

4: []

So nodes 1,3,4 are terminals.

So initial queue has nodes 1,3,4.

Processing node 1: reversed_adj[1] has u=0 (since 0→1). So for u=0, outdegree[0] is 2 (original edges to 1 and 2). Decrement by 1 → 1. Not zero.

Processing node 3: reversed_adj[3] has u=2. Decrement u=2's outdegree (which was 1) to 0. Add 2 to queue.

Processing node 4: reversed_adj[4] is empty, so nothing happens.

Queue now has 2.

Process 2: set safe[2] to True. reversed_adj[2] is u=0. Decrement u=0's outdegree from 1 to 0. Add 0 to queue.

Process 0: set safe[0] to True. reversed_adj[0] is empty. So queue is empty.

So the safe nodes are 0,1,2,3,4. But node 0's outgoing edges are to 1 (safe) and 2 (safe). So yes, all paths from 0 lead to safe nodes. So yes, correct.

So the code seems to handle these cases.

So the function would be implemented as above.

In [2]:
def find_eventual_safe_states(graph):
    n = len(graph)
    outdegree = [0] * n
    reversed_adj = [[] for _ in range(n)]
    
    for u in range(n):
        outdegree[u] = len(graph[u])
        for v in graph[u]:
            reversed_adj[v].append(u)
    
    queue = []
    for u in range(n):
        if outdegree[u] == 0:
            queue.append(u)
    
    safe = [False] * n
    while queue:
        v = queue.pop(0)
        safe[v] = True
        for u in reversed_adj[v]:
            outdegree[u] -= 1
            if outdegree[u] == 0:
                queue.append(u)
    
    return [u for u in range(n) if safe[u]]


args = dict(
    graph = [[1,2],[2,3],[5],[0],[5],[],[]]
)

find_eventual_safe_states(**args)

[2, 4, 5, 6]